In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import random
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import scipy.io



from sklearn.metrics import confusion_matrix
import glob
from torchvision.datasets.folder import IMG_EXTENSIONS
import time
import torch.nn.functional as F



In [ ]:
BATCH_SIZE = 200#100
BATCH_SIZEV = 200#560
BATCH_SIZEV2 = 600
EPOCH = 500
w = 3
in_w = 3
#
sqe = 12
# # of class
cls = 4
kk= 200


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
    
        self.cnn = nn.Sequential( 
        nn.Conv1d(in_w, 64, 9, stride=1, padding=4),
        nn.LeakyReLU(inplace=False),
        nn.Conv1d(64, 64, 9, stride=1, padding=4),
        nn.LeakyReLU(inplace=False),
        nn.Conv1d(64, 64, 9, stride=1, padding=4),
        #nn.LeakyReLU(inplace=False), #put it back 2020 706
        )
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        #embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]

        src1 = src.permute(0, 2, 1)
        #print src1.size()
        src2 = self.cnn(src1)
        #print src2.size()
        src3 = src2.permute(0, 2, 1)
        #print src3.size()

    
    
        return src3
    
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super(Attention, self).__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = nn.Linear((enc_hid_dim * 1) + dec_hid_dim, dec_hid_dim, bias=True)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        

        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        #encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]


        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        #print energy
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
     
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]

        
        return F.softmax(attention, dim=1)



class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super(Encoder, self).__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        #ORINAL 1 # 5
        self.rnn = nn.GRU(64, enc_hid_dim, 3, bidirectional = False, dropout=0.5, batch_first = True)
        
        self.fc = nn.Sequential( 
            nn.Linear(enc_hid_dim * 1, dec_hid_dim, bias=True),
            #nn.BatchNorm1d(dec_hid_dim),
            nn.Dropout(0.5),
        )
     
    def forward(self, src):
        
        #src = [src sent len, batch size]
        

        outputs, hidden = self.rnn(src)
  
        #hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        hidden = self.fc(hidden[-1,:,:])
        #hidden = self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        #outputs = [src sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        #print outputs
        return outputs, hidden


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super(Decoder, self).__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        #self.dropout = dropout
        self.attention = attention
        
        #self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 1) + emb_dim, dec_hid_dim,batch_first = True, dropout=0.0)
        
        self.out = nn.Sequential( 

            nn.Linear((enc_hid_dim * 1) + dec_hid_dim + emb_dim, output_dim, bias=False),
            #nn.Dropout(0.5),
        )
        
        self.dropout = nn.Dropout(dropout)


    def forward(self, input, hidden, encoder_outputs, trans):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        
        a = self.attention(hidden, encoder_outputs)
        #print a        
        #a = [batch size, src len]
    
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        encoder_outputs = encoder_outputs.permute(0, 1, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        

        rnn_input = torch.cat((input, weighted), dim = 2)
        rnn_input = rnn_input.permute(1, 0, 2)
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
        #print hidden.size()

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        #print output 
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden

        #assert (output == hidden).all()
        
        #input = input.squeeze(0)
        output = output.squeeze(1)
        output = output.unsqueeze(0)
        #weighted = weighted.squeeze(0)
        #print(output.size())
        #print(weighted.size())
        #print(input.size())
        #print(trans.size())
        #trans = trans.unsqueeze(0)
        #output = self.out(torch.cat((output, weighted, input, trans), dim = 2))
        output = self.out(torch.cat((output, weighted, input), dim = 2))
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device = 0):
        super(Seq2Seq, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        tn = scipy.io.loadmat("./transition.mat")
        trans = tn['transition_train_raw']
        self.trans = Variable(torch.FloatTensor(trans).cuda())
        #print(trans)
        #assert encoder.hid_dim == decoder.hid_dim, \
        #    "Hidden dimensions of encoder and decoder must be equal!"
        #assert encoder.n_layers == decoder.n_layers, \
        #    "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[0]
        max_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(batch_size, max_len-1, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        encoder_outputs, hidden = self.encoder(src)
        #print hidden.size()
        #first input to the decoder is the <sos> tokens
        input = trg[:,0]
        _, ind = torch.max(input,1)
        trans_bat = torch.zeros(batch_size, 5).to(self.device)
        for i in range(batch_size):
            trans_bat[i,:] = self.trans[ind[i],:]
        #print(input.size())
        #print(input2.size())
        
        for t in range(1, max_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden = self.decoder(input, hidden, encoder_outputs, trans_bat)

            #place predictions in a tensor holding predictions for each token
   
            if output.size()[0] == 1:
                outputs[:,t-1,:] = output[:,:]
            else:	
                outputs[:,t-1,:] = output[:,0,:]
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            #need to chekc again   tzuan
            top1 = output.argmax(1)
            top1 = top1.view(-1,1).type(torch.cuda.FloatTensor)
            #print "Tp: ", top1.size()
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[:,t] if teacher_force else output.squeeze(0)
        

        return outputs

class features_dataset(Data.Dataset):

    def __init__(self, mode, p):

        self.mode = mode
        self.data = glob.glob(p)

    def __getitem__(self, index):


        if self.mode == 0:
            data = scipy.io.loadmat(self.data[index])
            x = np.squeeze(data['sqe_x'])[:800,:]
            y = data['sqe_y'][:800,:]
        elif self.mode == 1:
            data = scipy.io.loadmat(self.data[index])
            x = np.squeeze(data['sqe_x'])
            y = data['sqe_y']
            up_lim = x.shape[0] - 700
            ind = random.randint(sqe, up_lim)
            tmp_x = x[(ind-sqe):ind+700,:]
            tmp_y = y[(ind-sqe):ind+700,:]
            x = tmp_x#.reshape((sqe+50, w)) + np.random.normal(0,0.000001, size=(50+sqe, w))
            y = tmp_y

        else:
            data = scipy.io.loadmat(self.data[index])
            x = np.squeeze(data['sqe_x'])
            y = data['sqe_y']
            tem_x = np.zeros((2953,3))
            tem_y = np.zeros((2953,1))
            tem_x[:x.shape[0],:] = x[:,[1,6,7]]
            tem_y[:y.shape[0],:] = y[:,:]
            x1 = torch.FloatTensor(tem_x) 
            y1 = torch.LongTensor(tem_y)
            y2 = torch.zeros(x1.size()[0],cls).scatter_(dim=1, index=y1, src=torch.tensor(1.0))
            
            return x1, y1, y2, x.shape[0]
        #x = self.input_x[index].reshape((h, w))
        #y = self.input_y[index]			
        #x = x[:,[0, 1, 6]]	
        x = x[:,[1,6,7]]
        x1 = torch.FloatTensor(x) 
        y1 = torch.LongTensor(y)#.unsqueeze(1)
        
        
        #print(y1.size(), "                    "  ,x1.size())

        #print(y1.size())
        y2 = torch.zeros(x1.size()[0],cls).scatter_(dim=1, index=y1, src=torch.tensor(1.0))
        #print y2.size()
        #quit()

        return x1, y1, y2
    def __len__(self):
        if self.mode == 1:
            return(len(self.data))
        else:
            return(len(self.data))
            #return(len(self.x))
    def getName(self):
        return self.classes

In [ ]:
path_d = "./data_focal5_t_noaug/*.mat"
train_loader = torch.utils.data.DataLoader(
    features_dataset(mode = 1, p=path_d),
    batch_size = BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    drop_last=True
)

path_d = "./data_focal5_t_noaug/*.mat"
train_loader2 = torch.utils.data.DataLoader(
    features_dataset(mode = 0, p=path_d),
    batch_size = BATCH_SIZEV2,
    shuffle=False,
    num_workers=4,
    drop_last=True
)

path_d = "./data_focal5_v/*.mat"
val_loader = torch.utils.data.DataLoader(
    features_dataset(mode = 2, p=path_d),
    batch_size = BATCH_SIZEV,
    shuffle=False,
    num_workers=4,
    drop_last=True
)


In [ ]:
INPUT_DIM = in_w
OUTPUT_DIM = cls
ENC_EMB_DIM = in_w
DEC_EMB_DIM = cls
ENC_DROPOUT = 0.50
DEC_DROPOUT = 0.50
device = 0
ENC_HID_DIM = 256
DEC_HID_DIM = 256
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
cnnmodel = CNN().to(device)
model = Seq2Seq(enc, dec, device).to(device)
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

#model.load_state_dict(torch.load(SAVE_PATH))
SAVE_PATH_CNN = "./params/616/CNN_4cls_part1.pkl"
SAVE_PATH_S2S = "./params/616/S2S_4cls_part2.pkl"
SAVE_PATH_CNN = "./params/628/CNN_4cls_part1_layer2_std_sqe_15.pkl"
SAVE_PATH_S2S = "./params/628/S2S_4cls_part2_layer2_std_sqe_15.pkl"
#model.load_state_dict(torch.load(SAVE_PATH_S2S))
#cnnmodel.load_state_dict(torch.load(SAVE_PATH_CNN))
#print("loaded!")
print(cnnmodel)
print(model)


MAX = 0.0


In [ ]:
class RCCLoss(nn.Module):
    def __init__(self, fn_weights=None, fp_weights=None):
        super(RCCLoss,self).__init__()
        #anti_eye = np.ones((k,k)) - np.eye(k)
        self.full_fn_weights = torch.Tensor(fn_weights)
        self.full_fp_weights = torch.Tensor(fp_weights)
        
    def forward(self, x, y):
        #print(x.size()[0])
        #print(y.size())
        #print(x)
        x = F.softmax(x, dim=1)
        yi = torch.cuda.LongTensor(y.unsqueeze(1))
        #print(yi)
        y1 = torch.zeros(x.size()[0],cls).cuda().scatter_(dim=1, index=yi, src=torch.tensor(1.0))
        #print(y1)
        if y1.type() != x.data.type():
            y1 = y1.type_as(x.data)
        eps=1e-5
        
        x = torch.clamp(x, eps, 1. - eps) 
        
        logs = torch.log(x)
        
        logs_1_sub = torch.log(1.-x) # shape (m, k), dense. 0 is good. 
        #logs =  F.log_softmax(x)
        
        #logs_1_sub =  F.log_softmax(1.-x) # shape (m, k), dense. 0 is good. 
        #print(logs_1_sub)
        if self.full_fn_weights.type() != x.data.type():
            self.full_fn_weights = self.full_fn_weights.type_as(x.data)

        if self.full_fp_weights.type() != x.data.type():
            self.full_fp_weights = self.full_fp_weights.type_as(x.data)
        
        m_full_fn_weights = torch.mm(y1, self.full_fn_weights) # (m,k) . (k, k)
        
        m_full_fp_weights = torch.mm(y1, self.full_fp_weights) # (m,k) . (k, k)
        #print(y1.type())
        #print(self.full_fn_weights.type())
        #print(m_full_fp_weights.type())
        loss = -(m_full_fn_weights * logs + m_full_fp_weights * logs_1_sub).sum()
        
        
        return loss


In [ ]:
# loss function
icf_sqrt = [
    1.6647,
    1.4874,
    3.8379,
    2.8959]
fn_weights =[
 [1.6647, 0.00, 0.00, 0.00],
 [0.00, 1.4874, 0.00, 0.00],
 [0.00, 0.00, 3.8379, 0.00],
 [0.00, 0.00, 0.00, 2.8959]]



fp_weights=[
 [0.00, 0.79, 0.17, 3.00],
 [0.79, 0.00, 12.00, 3.80],
 [0.17, 7.00, 0.00, 1.80],
 [3.00, 3.80, 1.80, 0.00]]

fp_weights2=[
 [0.00, 1.25, 5.88, 0.33],
 [0.79, 0.00, 0.14, 0.25],
 [0.17, 7.00, 0.00, 0.50],
 [3.00, 3.80, 1.80, 0.00]]

fp_weights3=[
 [0.00, 0.79, 0.17, 3],
 [1.25, 0.00, 7, 3.8],
 [5.88, 0.14, 0.00, 0.18],
 [0.33, 0.25, 0.5, 0.00]]

fp_weights4= [
 [0.00, 1.25**0.5, 5.88**0.5, 0.33**0.5],
 [0.79**0.5, 0.00, 0.14**0.5, 0.25**0.5],
 [0.17**0.5, 7.00**0.5, 0.00, 0.50**0.5],
 [3.00**0.5, 3.80**0.5, 1.80**0.5, 0.00]]

print(fp_weights4)
loss_func = RCCLoss(fn_weights=fn_weights, fp_weights=fp_weights4)
 
LR = 0.00015
print(loss_func)

In [ ]:
loss_t = np.zeros(EPOCH)
loss_v = np.zeros(EPOCH)
acc_t = np.zeros(EPOCH)
acc_v = np.zeros(EPOCH)
for epoch in range(EPOCH):

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    optimizer2 = torch.optim.Adam(cnnmodel.parameters(), lr=LR)
    model.train()
    cnnmodel.train()
    #training

    for step, (x, y, y2) in enumerate(train_loader):

        y3 = torch.zeros(y2.size())
        #y3[:,0:2,:] = y2[:,0:2,:]
        y3 = y2

        for ii in range(sqe, 700+sqe):
            #ii = 
            data_in = torch.FloatTensor(BATCH_SIZE, sqe, in_w)
            data_out = torch.LongTensor(BATCH_SIZE, 1,1)
            data_out3 = torch.LongTensor(BATCH_SIZE, sqe-1, 1)
            target = torch.FloatTensor(BATCH_SIZE, sqe, cls)
            target2 = torch.FloatTensor(BATCH_SIZE, sqe-1, cls)

            data_in[:,:,:] = x[:,ii-sqe:ii,:]
    
            data_out3[:,:,:] = y[:,ii-sqe+1:ii,:]
            data_out4 = data_out3.squeeze(2).reshape(BATCH_SIZE*(sqe-1))
            data_out5 = data_out3.squeeze(2).reshape(BATCH_SIZE,(sqe-1)).permute(1,0)

            target[:,:,:] = y3[:,ii-sqe:ii,:] 
            target2[:,:,:] = y2[:,ii-sqe+1:ii,:] 
            t_x = Variable(data_in.cuda())
            t_y2 = Variable(data_out4.cuda())
            t_y3 = Variable(data_out5.cuda())
            trg = Variable(target.cuda())
            trg2 = Variable(target2.cuda())
            tmp = cnnmodel(t_x)
            outputs = model(tmp,trg, 0.0)

            _, pred = torch.max(outputs[:,sqe-2,:].contiguous().view(-1,cls).data, 1)
            y_3 = torch.zeros(BATCH_SIZE,cls).scatter_(dim=1, index=pred.cpu().unsqueeze(1).data, src=torch.tensor(1.0))

            y3[:,ii-1,:] = outputs[:,sqe-2,:]

            
             
            #print(outputs)    
            loss = loss_func(outputs.contiguous().view(-1,cls), t_y2)
            #print(loss)
            optimizer.zero_grad()   # clear gradients for next train
            optimizer2.zero_grad()
            loss.backward()         # backpropagation, compute gradients 

            optimizer.step()        # apply gradients
            optimizer2.step()
    if (epoch + 1)%1 ==0:

        print("Training done!")
        model.eval()
        cnnmodel.eval()
        mse = np.zeros(BATCH_SIZEV2)
        pmse = np.zeros(BATCH_SIZEV2)
        count = 0.0
        acc = np.zeros(cls)
        total = np.zeros(cls)
        cf = np.zeros((cls,cls))

        for step, (x, y, y2) in enumerate(train_loader2):

            with torch.no_grad():

                y3 = torch.zeros(y2.size())
                y3[:,0:sqe,:] = y2[:,0:sqe,:]
                y5  = torch.zeros(y.size(),dtype=torch.long)
                y5[:,0:1,:] = y[:,0:1,:]
                y_mode = torch.zeros(y2.size(),dtype=torch.long)
                y_mode[:,0:sqe,:] = y2[:,0:sqe,:]
                for ii in range(sqe, 801):

                    data_in = torch.FloatTensor(BATCH_SIZEV2,sqe,in_w)
                    data_out = torch.LongTensor(BATCH_SIZEV2,sqe-1,1)
                    target = torch.FloatTensor(BATCH_SIZEV2, sqe, cls)
                    data_in[:,:,:] = x[:,ii-sqe:ii,:]
                    data_out[:,:,:] = y[:,ii-sqe+1:ii,:]

                    target[:,:,:] = y3[:,ii-sqe:ii,:]
                    data_out4 = data_out.squeeze(2).reshape(BATCH_SIZEV2*(sqe-1))
                    data_out = data_out.squeeze(2)

                    v_x = Variable(data_in.cuda())
                    trg = Variable(target.cuda())
                    t_y2 = Variable(data_out4.cuda())
                    tmp = cnnmodel(v_x)
                    outputs = model(tmp,trg, 0.0)
                    #loss = loss_func(outputs.contiguous().view(-1,cls), t_y2)
                    #loss_t[epoch] = loss 
                    y3[:,ii-1,:] = outputs[:,sqe-2,:]
                    _, pred2 = torch.max(outputs[:,sqe-2,:].contiguous().view(-1,cls).data, 1)
                    plabel = pred2.cpu().data
                    _, pred3 = torch.max(outputs.data, 2)

                    tt = torch.cat((y5[:,ii-sqe+1:ii,:],pred3.contiguous().view(BATCH_SIZEV2,sqe-1,1).cpu()),2)
                    tt3, tt2 = torch.max(tt,2)
                    y5[:,ii-sqe+1:ii,0] = tt3
                    y_m = pred3.contiguous().view(BATCH_SIZEV2,sqe-1,1).cpu()
                    for kk in range(BATCH_SIZEV):
                        y_mode[kk,ii-sqe+1:ii,:] += torch.zeros(outputs.size()[1],cls,dtype=torch.long).scatter_(dim=1, index=y_m[kk,:,:], src=torch.tensor(1.0))

                    for tt in range(BATCH_SIZEV2):
                        cf[data_out4[(tt+1)*(sqe-1)-1], plabel[tt]] +=1
                        total[data_out4[(tt+1)*(sqe-1)-1]] +=1

                        if plabel[tt] == data_out4[(tt+1)*(sqe-1)-1]:
                            acc[data_out4[(tt+1)*(sqe-1)-1]] += 1
                            count += 1
        acc_t[epoch] =count/total.sum()
        print("Train EPOCH : ", epoch + 1, " ACC : ", count/total.sum(), "    LR: ", LR)
        print("ACC 0 wake : ", acc[0]/total[0], " ACC 1 stage 1 : ", acc[1]/total[1], " ACC 2 stage 2 : ", acc[2]/total[2], " ACC 3 stage 3 : ", acc[3]/total[3])
        #print("Train Loss: ", loss_t[epoch])
        print(total[0], "   ", total[1], "   ", total[2] , "   ", total[3])
        np.set_printoptions(precision=3,suppress=True)
        print("Train Confusion matrix : ")
        print(cf)

        mse = np.zeros(BATCH_SIZEV)
        pmse = np.zeros(BATCH_SIZEV)
        count = 0.0
        acc = np.zeros(cls)
        total = np.zeros(cls)
        cf = np.zeros((cls,cls))

        for step, (x, y, y2, lens) in enumerate(val_loader):

            with torch.no_grad():


                #mem = torch.FloatTensor(BATCH_SIZEV, 600, cls)
                y3 = torch.zeros(y2.size())
                y3[:,0:sqe,:] = y2[:,0:sqe,:]
                y5  = torch.zeros(y.size(),dtype=torch.long)
                y5[:,0:1,:] = y[:,0:1,:]
                y6  = torch.zeros(y.size(),dtype=torch.long)
                y6[:,0:sqe,:] = y[:,0:sqe,:]
                y_mode = torch.zeros(y2.size(),dtype=torch.long)
                y_mode[:,0:sqe,:] = y2[:,0:sqe,:]

                
                for ii in range(sqe, 2953):
                    data_in = torch.FloatTensor(BATCH_SIZEV,sqe,in_w)
                    data_out = torch.LongTensor(BATCH_SIZEV,sqe-1,1)
                    target = torch.FloatTensor(BATCH_SIZEV, sqe, cls)

                    data_in[:,:,:] = x[:,ii-sqe:ii,:]
                    data_out[:,:,:] = y[:,ii-sqe+1:ii,:]

                    target[:,:,:] = y3[:,ii-sqe:ii,:]
                    data_out4 = data_out.squeeze(2).reshape(BATCH_SIZEV*(sqe-1))
                    data_out = data_out.squeeze(2)
                    v_x = Variable(data_in.cuda())
                    #v_y = Variable(data_out.cuda())
                    trg = Variable(target.cuda())
                    t_y2 = Variable(data_out4.cuda())
                    tmp = cnnmodel(v_x)
                    outputs = model(tmp,trg, 0.0)
                    #loss = loss_func(outputs.contiguous().view(-1,cls), t_y2)
                    #loss_v[epoch] = loss 
                    y3[:,ii-1,:] = outputs[:,sqe-2,:]

                    _, pred2 = torch.max(outputs[:,sqe-2,:].contiguous().view(-1,cls).data, 1)
                    plabel = pred2.cpu().data

                    _, pred3 = torch.max(outputs.data, 2)

                    tt = torch.cat((y5[:,ii-sqe+1:ii,:],pred3.contiguous().view(BATCH_SIZEV,sqe-1,1).cpu()),2)
                    tt3, tt2 = torch.max(tt,2)
                    y5[:,ii-sqe+1:ii,0] = tt3                    
                    y_m = pred3.contiguous().view(BATCH_SIZEV,sqe-1,1).cpu()
                    for kk2 in range(BATCH_SIZEV):
                        y_mode[kk2,ii-sqe+1:ii,:] += torch.zeros(outputs.size()[1],cls,dtype=torch.long).scatter_(dim=1, index=y_m[kk2,:,:], src=torch.tensor(1.0))

                    for tt in range(BATCH_SIZEV):

                        cf[data_out4[(tt+1)*(sqe-1)-1], plabel[tt]] +=1

                        total[data_out4[(tt+1)*(sqe-1)-1]] +=1
                        if plabel[tt] == data_out4[(tt+1)*(sqe-1)-1]:
                            acc[data_out4[(tt+1)*(sqe-1)-1]] += 1
                            count += 1



        mse = np.zeros(BATCH_SIZEV)
        pmse = np.zeros(BATCH_SIZEV)
        count = 0.0
        acc = np.zeros(cls)
        total = np.zeros(cls)
        cf = np.zeros((cls,cls))
        _, y8 = torch.max(y_mode,2)
        for i in range(BATCH_SIZEV):
            l = lens[i]
            for j in range(l):
                cf[y[i,j,0], y8[i,j]] +=1
                total[y[i,j,0]] +=1
                if y8[i,j] == y[i,j,0]:
                    acc[y[i,j,0]] += 1
                    count += 1
        acc_v[epoch] = count/total.sum()                   
        print("Val Mode EPOCH : ", epoch + 1, " ACC : ", count/total.sum(), "    LR: ", LR)
        print("ACC Wake : ", acc[0]/total[0], " ACC Light : ", acc[1]/total[1], " ACC Deep : ", acc[2]/total[2], " ACC REM : ", acc[3]/total[3])
        print("Val loss: ", loss_v[epoch])
        print(total[0], "   ", total[1], "   ", total[2] , "   ", total[3])

        np.set_printoptions(precision=3,suppress=True)
        print("Val Mode Confusion matrix : ")
        print(cf)
        # saving models
        SAVE_PATH_CNN = "./params/N800/CE_3features_rrc_new_fp_4_sqrt_ICF_tas/CNN_4cls_part1_layer3_std_sqe_12_Epoch_" + str(epoch + 1) + ".pkl"
        SAVE_PATH_S2S = "./params/N800/CE_3features_rrc_new_fp_4_sqrt_ICF_tas/S2S_4cls_part2_layer3_std_sqe_12_Epoch_" + str(epoch + 1) + ".pkl"
        torch.save(cnnmodel.state_dict(), SAVE_PATH_CNN)
        torch.save(model.state_dict(), SAVE_PATH_S2S)
        print("Model saved!!")